In [2]:
# time series model for prediction of next dat temperature

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense
from sklearn.model_selection import train_test_split

In [3]:
# Load the dataset into the dataframe
# parse_date : is used to convert the date column into datetime objects
df = pd.read_csv("daily_minimum_temps.csv",parse_dates = ['Date'],index_col='Date')

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_16816\1747212286.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv("daily_minimum_temps.csv",parse_dates = ['Date'],index_col='Date')


In [4]:
# checking the first
df.head()

,Temp
Date,
1981-01-01,20.7
1981-01-02,17.9
1981-01-03,18.8
1981-01-04,14.6
1981-01-05,15.8


In [5]:
df.isnull().sum()


Temp    0
dtype: int64

In [6]:
df.dtypes

Temp    object
dtype: object

In [7]:
df.shape

(3650, 1)

In [8]:
from math import nan
# string format for a numric value can be
a='1'
stock = "one hundred three"
stock = nan
temp = "hello" # no sense
temp = nan

In [9]:
# converting the string values to numeric values by removing the double quotes and if string value does bot make any sense or it
# we convert that value to null
df['Temp'] = pd.to_numeric(df['Temp'],errors = 'coerce')

In [10]:
df.isnull().sum()

Temp    3
dtype: int64

In [11]:
df = df.dropna()

In [12]:
df.isnull().sum()

Temp    0
dtype: int64

In [13]:
from sklearn.preprocessing import MinMaxScaler

# Ensure 'Temp' is a column and not a method
print(df.columns)  # Should include 'Temp'

# Apply MinMax scaling
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(df['Temp'].values.reshape(-1, 1))


Index(['Temp'], dtype='object')


In [14]:
#
seq_length = 30

In [15]:
# fucttion for creating sequence
def create_sequences(data_scaled,seq_length):
  X,y =[],[]
  for i in range(len(data_scaled)-seq_length):
    X.append(data_scaled[i:i+seq_length])
    y.append(data_scaled[i+seq_length])
  return np.array(X),np.array(y)



In [16]:
# create the function and storing the value in list X and y
X,y = create_sequences(data_scaled,seq_length)

In [17]:
# Divide the dataset into the train test and split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,shuffle = False)


In [18]:
#building the rnn model
model = Sequential([
    LSTM(64,activation = "relu",input_shape = (seq_length,1)),
    Dense(1)
])

In [19]:
from tensorflow.keras.losses import MeanSquaredError
model.compile(optimizer='adam', loss=MeanSquaredError())


In [20]:
# Train the model
model.fit(X_train,y_train,epochs =20,batch_size=32)

Epoch 1/20

91/91 [==============================] - 2s 10ms/step - loss: 0.0242
Epoch 2/20
91/91 [==============================] - 1s 11ms/step - loss: 0.0107
Epoch 3/20
91/91 [==============================] - 1s 9ms/step - loss: 0.0105
Epoch 4/20
91/91 [==============================] - 1s 9ms/step - loss: 0.0104
Epoch 5/20
91/91 [==============================] - 1s 8ms/step - loss: 0.0103
Epoch 6/20
91/91 [==============================] - 1s 8ms/step - loss: 0.0100
Epoch 7/20
91/91 [==============================] - 1s 8ms/step - loss: 0.0099
Epoch 8/20
91/91 [==============================] - 1s 8ms/step - loss: 0.0097
Epoch 9/20
91/91 [==============================] - 1s 8ms/step - loss: 0.0095
Epoch 10/20
91/91 [==============================] - 1s 7ms/step - loss: 0.0093
Epoch 11/20
91/91 [==============================] - 1s 7ms/step - loss: 0.0093
Epoch 12/20
91/91 [==============================] - 1s 7ms/step - loss: 0.0091
Epoch 13/20
91/91 [===========================

In [21]:
y_pred_scaled = model.predict(X_test)

23/23 [==============================] - 0s 6ms/step


In [22]:
#inverse transform the scaled data

y_pred_scaled = np.clip(y_pred_scaled,0,1)

y_pred = scaler.inverse_transform(y_pred_scaled)

y_test_actual = scaler.inverse_transform(y_test)

In [23]:
last_sequence = data_scaled[-seq_length:].reshape(1, seq_length, 1)
next_temp_scaled = model.predict(last_sequence)
next_temp_scaled = np.clip(next_temp_scaled, 0, 1)
next_day_temp = scaler.inverse_transform(next_temp_scaled)

print(f"Next day's temperature is : ", next_day_temp[0][0])

1/1 [==============================] - 0s 27ms/step
Next day's temperature is :  12.741159


In [24]:
model.save("model.h5")


c:\Users\LENOVO\anaconda3\envs\ml_env\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [25]:
import joblib
joblib.dump(scaler, "scaler.pkl")


['scaler.pkl']